# Raman spectrum baseline subtraction and peak fitting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import peakutils
from scipy.signal import find_peaks

%matplotlib tk

: 

In [2]:
data_file_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\MWCNTs_patterned.txt"
df = pd.read_csv(data_file_path, sep=r"\s+", skiprows=1, header=None)
data = np.genfromtxt(data_file_path, delimiter="\t", skip_header=1)
wave_numbers = data[:, 0]
intensities = data[:, 1]
print(wave_numbers)

[1967.354492 1965.749023 1964.142578 ...  118.995117  116.893555
  114.791992]


In [5]:
# print number of elements in the find_peaks tuble
peak_indices = find_peaks(intensities, prominence= 500)[0]
print(peak_indices[0], peak_indices[1])
plt.plot(wave_numbers, intensities)
plt.scatter(wave_numbers[peak_indices], intensities[peak_indices], c='r')
plt.show()
print(intensities[peak_indices[0]],intensities[peak_indices[1]])

print('I_D/I_G = ', intensities[peak_indices[1]]/intensities[peak_indices[0]])


234 380
6162.945313 1059.322876
I_D/I_G =  0.17188581468757874


In [5]:
baseline_values = peakutils.baseline(intensities, 2)
plt.plot(df[0], df[1])
plt.plot(wave_numbers, baseline_values, color="red")
plt.show()

In [7]:
import glob

my_files_path = glob.glob(r'C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\*.txt')
print(my_files_path)

['C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_patterned.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_patterned1.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_patterned2.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_patterned_map_bottom.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_patterned_map_Copy.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_unpatterned.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_unpatterned1.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Data\\Midi\\Raman\\23.06.21 MWCNTs\\MWCNTs_unpatterned2.txt', 'C:\\Users\\lb958\\OneDrive - University of Cambridge\\Da

In [6]:
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [9]:
def baseline_als(y, lam, p, niter=10):
  L = len(y)
  D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
  w = np.ones(L)
  for i in range(niter):
    W = sparse.spdiags(w, 0, L, L)
    Z = W + lam * D.dot(D.transpose())
    z = spsolve(Z, w*y)
    w = p * (y > z) + (1-p) * (y < z)
  return z

In [10]:
baseline = baseline_als(intensities, 100000, 0.0001, 1000)
plt.plot(wave_numbers, intensities)
plt.plot(wave_numbers, baseline)

In [10]:
from pybaselines import Baseline, utils

def generate_baseline(x_data, y_data):
    baseline_fitter = Baseline(x_data = x_data)
    baseline = baseline_fitter.mor(y_data, half_window=30)[0]
    return baseline



### Analysis from file to baseline subtraction + I_D/I_G

In [8]:
def return_DtoG_ratio(data):
    if type(data) == str:
        data = np.genfromtxt(data, delimiter="\t", skip_header=1)
    wave_numbers = data[:, 0]
    intensities = data[:, 1]
    baseline = generate_baseline(wave_numbers, intensities)
    corrected_intensities = intensities - baseline
    corrected_intensities = corrected_intensities/np.max(corrected_intensities)

    peak_indices = find_peaks(corrected_intensities, prominence= 0.05, distance=20, width=10)[0]
    #discard peak indices in first half of spectrum
    peak_indices = peak_indices[peak_indices < len(wave_numbers)/2]
    try:
        I_D_I_G = corrected_intensities[peak_indices[1]]/corrected_intensities[peak_indices[0]]
        return I_D_I_G
    except IndexError:
        return None
    

In [7]:
def process_raman_data(wave_numbers,intensities, fig, ax):
    # generate baseline
    baseline = generate_baseline(wave_numbers, intensities)
    corrected_intensities = intensities - baseline
    corrected_intensities = corrected_intensities/np.max(corrected_intensities)
    #plot raw data, baseline and corrected data
    ax.set_title('Raman spectrum')
    ax.set_xlabel('Wave number (cm$^{-1}$)')
    ax.set_ylabel('Intensity (a.u.)')
    
    ax.plot(wave_numbers, corrected_intensities)
    
    #remove numbers on y axis
    ax.set_yticklabels([])

    # find and plot peaks
    peak_indices = find_peaks(corrected_intensities, prominence= 0.05, distance=20, width=10)[0]
    #discard peak indices in first quarter of spectrum
    peak_indices = peak_indices[peak_indices < len(wave_numbers)/2]

    ax.scatter(wave_numbers[peak_indices], corrected_intensities[peak_indices], c='r')
    # mark peak positions on plot
    peaks_string = ""
    for i in peak_indices:
        peaks_string = peaks_string + "Peak at {:.2f} cm$^{{-1}}$, intensity {:.2f} \n".format(wave_numbers[i], corrected_intensities[i])
    # place box with peaks_string
    ax.text(0.05, 0.5, peaks_string, 
            transform=ax.transAxes, fontsize=10, ha='left', va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

    # place box with I_D/I_G value
    ax.text(0.05, 0.95, 'I_D/I_G = {:.4f}'.format(corrected_intensities[peak_indices[1]]/corrected_intensities[peak_indices[0]]), 
            transform=ax.transAxes, fontsize=14, ha='left', va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))    

In [8]:
def process_raman(data_file_path,figure_generated = False):
    if figure_generated == False:
        fig, ax = plt.subplots()
    # generate data from file
    data = np.genfromtxt(data_file_path, delimiter="\t", skip_header=1)
    wave_numbers = data[:, 0]
    intensities = data[:, 1]

    process_raman_data(wave_numbers,intensities, fig, ax)
    fig.show()



In [59]:
process_raman(data_file_path)
print(np.genfromtxt(data_file_path, delimiter="\t", skip_header=1).shape)
# print(return_DtoG_ratio(np.genfromtxt(data_file_path, delimiter="\t", skip_header=1)))

(1011, 2)


In [60]:
MWCNT_unpatterned_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\MWCNTs_unpatterned1.txt"
process_raman(MWCNT_unpatterned_path)

In [5]:
map_data_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\MWCNTs_patterned_map_bottom.txt"
# Read the data from the file
data = pd.read_csv(map_data_path, sep=r'\s+')
data.head()


,#X,#Y,#Wave,#Intensity
0,-120.0,-80.0,1879.449219,1078.561768
1,-120.0,-80.0,1877.822266,1026.323730
2,-120.0,-80.0,1876.194336,1092.124634
3,-120.0,-80.0,1874.566406,1101.073730
4,-120.0,-80.0,1872.938477,1116.159546


In [3]:
def clickable_raman_map(data_file_path):
    data = pd.read_csv(data_file_path, sep=r'\s+')
    # Extract x, y coordinates, wavelength, and intensity
    x_coordinates = data['#X']
    y_coordinates = data['#Y']
    wavelengths = data['#Wave']
    intensities = data['#Intensity']

    # Create a grid plot
    fig, ax = plt.subplots()

    # Plot the grid points
    ax.scatter(x_coordinates, y_coordinates)

    # Function to handle mouse click events
    def on_click(event):
        if event.inaxes is not None:
            # Retrieve the clicked point coordinates
            x = 5*(round(event.xdata/5))
            y = 5*(round(event.ydata/5))
            
            # Filter the data for the clicked point
            mask = (x_coordinates == x) & (y_coordinates == y)
            point_wavelengths = np.array(wavelengths[mask])
            point_intensities = np.array(intensities[mask])
            
            # Create a new figure for the spectrum plot
            fig_spectrum, ax_spectrum = plt.subplots()
            
            # Plot the spectrum
            process_raman_data(point_wavelengths, point_intensities, fig_spectrum, ax_spectrum)
            
            # Show the spectrum plot
            plt.show()


    # Connect the click event to the function
    fig.canvas.mpl_connect('button_press_event', on_click)

    plt.show()


In [11]:
clickable_raman_map(map_data_path)

In [4]:
def extract_spectrum(map_data_path, x, y):
    spectrum = []
    with open(map_data_path, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:
            values = line.strip().split('\t')
            x_value = float(values[0])
            y_value = float(values[1])
            if x_value == x and y_value == y:
                wavelength = float(values[2])
                intensity = float(values[3])
                spectrum.append((wavelength, intensity))
    return np.array(spectrum)

In [13]:
def DtoG_ratio_map(map_data_path, saveas = None, transparent = False, textcolour = None):
    # Load the data from the file
    data = np.loadtxt(map_data_path, skiprows=1)  # Skip the header row
    # Extract the relevant columns
    # making negative as map is upside down relative to microscope image
    x = data[:, 0]
    y = data[:, 1]
    print(x)
    I_D_I_G = []
    # dictionary to store the tested points and their I_D/I_G ratios
    tested_x_y = {}
    # extract spectrum for each point and calculate I_D/I_G
    for i in range(len(x)):
        if (x[i], y[i]) not in tested_x_y.keys():
            #extracts spectrum for the point and gets array of wave numbers and intensities
            spectrum = extract_spectrum(map_data_path, x[i], y[i])
            wave_numbers = np.array([spectrum[j][0] for j in range(len(spectrum))])
            intensities = np.array([spectrum[j][1] for j in range(len(spectrum))])
            #calculates I_D/I_G ratio for that spectrum
            intensity_ratio = return_DtoG_ratio(np.vstack((wave_numbers, intensities)).T)
            #adds the ratio to the dict
            tested_x_y[(x[i], y[i])] = intensity_ratio
        else:
            continue
    if type(textcolour) == str:
            matplotlib.rcParams['text.color'] = textcolour
            matplotlib.rcParams['axes.labelcolor'] = textcolour
            matplotlib.rcParams['xtick.color'] = textcolour
            matplotlib.rcParams['ytick.color'] = textcolour
    else:
        matplotlib.rcParams['text.color'] = 'black'
        matplotlib.rcParams['axes.labelcolor'] = 'black'
        matplotlib.rcParams['xtick.color'] = 'black'
        matplotlib.rcParams['ytick.color'] = 'black'      
        # plot the map
    plt.scatter([key[0] for key in tested_x_y.keys()], [key[1] for key in tested_x_y.keys()], c=[value for value in tested_x_y.values()], cmap='jet')
    # add title for colorbar
    plt.colorbar().set_label('I$_D$/I$_G$')
    # add title for the plot
    plt.title('I$_D$/I$_G$ map')
    plt.gca().invert_yaxis()
    # add labels for the axes
    plt.xlabel('x (µm)')
    plt.ylabel('y (µm)')
    
    # include spots with no peaks in black
    plt.scatter([key[0] for key in tested_x_y.keys() if tested_x_y[key] == None], [key[1] for key in tested_x_y.keys() if tested_x_y[key] == None], c='black')
    
    plt.show()
    if type(saveas) == str:
        plt.savefig(saveas, transparent = transparent)
   


In [58]:
DtoG_ratio_map(map_data_path, saveas = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\DtoG_map_bottom.png", transparent = False)

[-120. -120. -120. ...  120.  120.  120.]


In [57]:
DtoG_ratio_map(map_data_path, saveas = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\DtoG_map_bottom_transparent.png", transparent = True, 
               textcolour='lime')

[-120. -120. -120. ...  120.  120.  120.]


In [62]:
MWCNT_unpatterned_map_data_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\MWCNTs_unpatterned_map.txt"
DtoG_ratio_map(MWCNT_unpatterned_map_data_path, saveas = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\unpatterned_DtoG_map.png", transparent = False)

[-150. -150. -150. ...  150.  150.  150.]


In [63]:
DtoG_ratio_map(MWCNT_unpatterned_map_data_path, saveas = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\unpatterned_DtoG_map_transparent.png", transparent = True,
               textcolour='lime')

[-150. -150. -150. ...  150.  150.  150.]


In [32]:
def D_to_G_ratio_map_stats(map_data_path, print_stats = True):
    data = np.loadtxt(map_data_path, skiprows=1)  # Skip the header row
    # Extract the relevant columns
    # making negative as map is upside down relative to microscope image
    x = data[:, 0]
    y = data[:, 1]
    print(x)
    I_D_I_G = []
    # dictionary to store the tested points and their I_D/I_G ratios
    tested_x_y = {}
    # extract spectrum for each point and calculate I_D/I_G
    for i in range(len(x)):
        if (x[i], y[i]) not in tested_x_y.keys():
            #extracts spectrum for the point and gets array of wave numbers and intensities
            spectrum = extract_spectrum(map_data_path, x[i], y[i])
            wave_numbers = np.array([spectrum[j][0] for j in range(len(spectrum))])
            intensities = np.array([spectrum[j][1] for j in range(len(spectrum))])
            #calculates I_D/I_G ratio for that spectrum
            intensity_ratio = return_DtoG_ratio(np.vstack((wave_numbers, intensities)).T)
            #adds the ratio to the dict
            tested_x_y[(x[i], y[i])] = intensity_ratio
        else:
            continue
    ratio_array = np.array([intensity for intensity in list(tested_x_y.values()) if intensity != None])
    print(ratio_array)
    if print_stats == True:
        print('Mean I_D/I_G ratio: {:.4f}'.format(ratio_array.mean()))
        print('Standard deviation of I_D/I_G ratio: {:.4f}'.format(ratio_array.std()))
        
    return ratio_array.mean(), ratio_array.std()

In [33]:
MWCNT_unpatterned_map_data_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Raman\23.06.21 MWCNTs\MWCNTs_unpatterned_map.txt"
D_to_G_ratio_map_stats(MWCNT_unpatterned_map_data_path)
D_to_G_ratio_map_stats(map_data_path)

[-150. -150. -150. ...  150.  150.  150.]
[0.13243243 0.12423647 0.26449375 0.12317153 0.11386892 0.08251268
 0.11773701 0.12239131 0.09469756 0.11181855 0.12381147 0.09641554
 0.08051998 0.14326142 0.13848135 0.12281462 0.11340885 0.1074007
 0.12431877 0.14627266 0.13106193 0.09100588 0.12430041 0.11850988
 0.11473645 0.12963358 0.15077758 0.09877808 0.08027526 0.10023429
 0.11656313 0.08068448 0.11320005 0.12880808 0.16045943]
Mean I_D/I_G ratio: 0.1207
Standard deviation of I_D/I_G ratio: 0.0316
[-120. -120. -120. ...  120.  120.  120.]
[0.28424319 0.35195505 0.28544944 0.25561041 0.2639323  0.32966649
 0.44875004 0.3246068  0.29480188 0.27390843 0.33818125 0.35488048
 0.32562259 0.28077398 0.32246387 0.40194791 0.46617414 0.35165417
 0.28422333 0.27030928 0.32336129 0.34946821 0.24992157 0.25354744
 0.29109457 0.2964516  0.29547141 0.3310462  0.28846463 0.24760001
 0.24223843 0.25819625 0.25261052 0.36258924 0.36491061 0.24935141
 0.25495085 0.28250436 0.29128087 0.27743164 0.29316

(0.293524082978037, 0.053245873054007554)